### Script to
1. Get CCLE and GDSC curve categories  from previous task file and assign DW (2) label to pairs for which ic50<max_dosage 
so that we can Check if Hump and rising curves are outliers (to be removed) or is it the property of some dr/cl ? 

(if it is the prop, then same pairs would be Hump in both datasets/ Rising in both datasets)
###### **NOTE: this script used dose response files in the old format : cl,dr have dataset specific names, 1 row for 1 pair not 1 doasge

In [9]:
__author__ = 'Aanchal'

In [1]:
import numpy as np
import pandas as pd
import pickle
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import FUNCTIONS as ff

In [2]:
import warnings
warnings.filterwarnings('ignore')


In [3]:
# define these 4 parameters based on which method you want to use to separate normal shape category 1) IC50 2) code
ccle_max_dose=3 #log2(8)
categoryMatrix_ccle=pd.read_csv("../data_processed/CurveCategoryDetection/"+"CCLE"+"_CurveCategory.tsv", sep='\t')
categoryMatrix_gdsc=pd.read_csv("../data_processed/CurveCategoryDetection/"+"GDSE"+"_CurveCategory.tsv", sep='\t')


In [4]:
categoryMatrix_ccle

,Unnamed: 0,17-AAG,AEW541,AZD0530,AZD6244,Erlotinib,Irinotecan,L-685458,LBW242,Lapatinib,...,PHA-665752,PLX4720,Paclitaxel,Panobinostat,RAF265,Sorafenib,TAE684,TKI258,Topotecan,ZD-6474
0,1321N1_CENTRAL_NERVOUS_SYSTEM,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1,22RV1_PROSTATE,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2,42MGBA_CENTRAL_NERVOUS_SYSTEM,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0
3,5637_URINARY_TRACT,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
4,639V_URINARY_TRACT,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
5,697_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
6,769P_KIDNEY,1.0,1.0,1.0,1.0,1.0,1.0,1.0,3.0,1.0,...,1.0,3.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
7,786O_KIDNEY,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0
8,8305C_THYROID,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
9,8505C_THYROID,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


##### Get (dataset specific) names of common pairs

In [22]:
with open("../data_processed/Objects/CCLE_doseResponses.obj","rb") as f:
    names_ccle=pickle.load(f)
with open("../data_processed/Objects/GDSC_doseResponses.obj","rb") as f:
    names_gdsc=pickle.load(f) 
    response_gdsc=pickle.load( f)
    logDosage_gdsc=pickle.load( f)
   

In [23]:
names_gdsc_temp=names_gdsc

In [24]:
 
common_CL_info = pd.read_excel('../data/drug_response/GDSC-CCLE-CTRP_conversion.xlsx', sheetname='Cell lines')
common_DR_info = pd.read_excel('../data/drug_response/GDSC-CCLE-CTRP_conversion.xlsx', sheetname='Drugs_edited')

Intersection=ff.getCommonPairs_CCLEnomenclature(names_ccle,names_gdsc_temp, common_CL_info,common_DR_info)

In [28]:
Intersection

,Cell_line_name,Drug _Name
0,A673_BONE,17-AAG
1,A673_BONE,PLX4720
2,A673_BONE,Nutlin-3
3,A673_BONE,PD-0332991
4,A673_BONE,PD-0325901
5,A673_BONE,AZD6244
6,SKES1_BONE,Erlotinib
7,SKES1_BONE,PHA-665752
8,SKES1_BONE,Paclitaxel
9,SKES1_BONE,Sorafenib


In [6]:
### splitting normal shape category using IC50
ic50_ccle=pd.read_csv('../data/drug_response/CCLE/ccle_all_abs_ic50_bayesian_sigmoid.csv') 
ic50_gdsc=pd.read_csv('../data/drug_response/GDSC/gdsc_all_abs_ic50_bayesian_sigmoid.csv') 

for i in range(len(common_CL_info.values[:,0])):
    gid=common_CL_info.values[i,0]
    names_gdsc['Cell_line_name']=(names_gdsc['Cell_line_name'].replace( gid  ,common_CL_info.values[i,2]) )

for i in range(len(common_DR_info.values[:,0])):
    gid=common_DR_info.values[i,0]
    names_gdsc['Drug _Name']=(names_gdsc['Drug _Name'].replace( gid  ,common_DR_info.values[i,3]) )
 
for i in range(len(Intersection)):
    
    ind_cl=np.where( categoryMatrix_ccle.values[:,0] == Intersection.values[i,0]  )[0]
    ind_dr=np.where( categoryMatrix_ccle.columns == Intersection.values[i,1] )[0]
    ccle_label= categoryMatrix_ccle.values[ind_cl , ind_dr  ]
    Intersection.loc[i,'ccle_category'] =  ccle_label
    
    if (ccle_label==1): ##not in prev code
        ind_cl=np.where( ic50_ccle.values[:,0] == Intersection.values[i,0]  )[0]
        ind_dr=np.where( ic50_ccle.columns == (Intersection.values[i,1]) )[0]
        ccle_ic50= ic50_ccle.values[ind_cl , ind_dr  ]
        if (ccle_ic50 < ccle_max_dose):
            Intersection.loc[i,'ccle_category'] = 2
        
        
    colno=0  
    gdsc_cl_id=common_CL_info.values[ common_CL_info['CCLE name']==Intersection.values[i,0] ,colno] 
    ind_cl=np.where( categoryMatrix_gdsc.values[:,0] == gdsc_cl_id )[0]
    gdsc_dr_id=common_DR_info.values[ common_DR_info['CCLE name']==Intersection.values[i,1] ,colno] 
    #print(Intersection.values[i,1], gdsc_dr_id)
    ind_dr=np.where( categoryMatrix_gdsc.columns ==str(gdsc_dr_id[0]) )[0]
    gdsc_label= categoryMatrix_gdsc.values[ind_cl , ind_dr  ]
    Intersection.loc[i,'gdsc_category'] =  gdsc_label
    
    if (gdsc_label==1):
        colno=0    
        gdsc_cl_id=common_CL_info.values[ common_CL_info['CCLE name']==Intersection.values[i,0] ,colno] 
        ind_cl=np.where( ic50_gdsc.values[:,0] == gdsc_cl_id )[0]

        gdsc_dr_id=common_DR_info.values[ common_DR_info['CCLE name']==Intersection.values[i,1] ,colno] 
        ind_dr=np.where( ic50_gdsc.columns ==str(gdsc_dr_id[0]) )[0]


        a=names_gdsc.index[   names_gdsc['Cell_line_name']== Intersection.values[i,0] ]
        b=names_gdsc.index[   names_gdsc['Drug _Name']==Intersection.values[i,1] ]
        ind2=names_gdsc.index[a & b][0]
        gdsc_max_dose=max(logDosage_gdsc.values[ind2, 0:9] )    
        gdsc_ic50= ic50_gdsc.values[ind_cl , ind_dr  ]

        if (gdsc_ic50 < gdsc_max_dose):
            Intersection.loc[i,'gdsc_category'] = 2


In [7]:
Intersection.to_csv('../data_processed/CurveCategoryDetection/Intersection_withCurveCategory_SplitNormalUsingIC50.csv',index=False)
#Intersection_ic50=pd.read_csv('../data_processed/CurveCategoryDetection/Intersection_withCurveCategory_SplitNormalUsingIC50.csv')


In [ ]:
Intersection_ic50=Intersection

In [30]:
a=2
b=a
b=b+2
b

4